# 문장생성을 위한 pretrained model
* pretrained language model 
    * GPT
    * ![GPT](img/GPT.png)
    * [그림 출처](https://ai.stackexchange.com/questions/27038/why-does-gpt-2-exclude-the-transformer-encoder)
    * GPT는 앞 컨텍스트(이전 단어들)가 주어졌을 때 다음 단어를 맞히는 방식으로 프리트레인 수행
* 문장 생성: 이전 단어들, 즉 context가 주어졌을 때 다음 단어로 어떤 단어가 오는 게 적절한지 분류하는 방식.
    * 입력: Context
    * 출력: 컨텍스트 다음 토큰의 등장 확률
* 사용할 모델: KoGPT2
* Huggingface에 등록된 모델이므로 Huggingface 라이브러리로 불러올 수 있다.

### 문장생성 task를 수행할 때 사용하는 기법
* 출처 ["How to generate text: using different decoding methods for language generation with Transformers"](https://huggingface.co/blog/how-to-generate)
* **그리디 서치**
    * 문장 생성 태스크: 이전 단어들을 기반으로 다음에 올 토큰의 등장 확률 계산.
    * 그러나 다음 단어를 선택할 때마다 어휘 집합 크기만큼의 경우의 수가 발생하면 계산 불가능. 매순간 가장 높은 확률값을 선택하는 방식.
    * ![greedsearch](img/greedy_search.png)
* **빔 서치**:
    * 그리디 서치에서 놓치는 토큰이 있을 수 있음. 빔 크기만큼의 선택지를 계산 범위에 넣는다. `num_beams` 파라미터로 조정 가능.
    * ![beemsearch](img/beam_search.png)
* **샘플링 서치**:
    * 위의 두 방식은 모델이 출력한 다음 토큰 확률 분포를 활용하는 방식이므로 여러 번 반복해도 그 결과는 동일하다.
    * 탑-k 샘플링은 모델이 예측한 다음 토큰 확률 분포에서 확률값이 가장 높은 k개 토큰 가운데 하나를 다음 토큰으로 선택하는 기법이다.
    * 샘플링 방식으로 다음 토큰을 선택하는 경우 같은 컨텍스트라 하더라도 수행 때마다 문장 생성 결과가 다르다.
    * `top_k` 파라미터로 조정 가능함.
    * ![samplingsearch](img/sampling_search.png)
* **샘플링 서치 with 템퍼러처 스케일링**
    * 모델의 다음 토큰 확률 분포를 대소 관계의 역전 없이 분포의 모양만을 바꾸어 다양한 문장을 생성하게 하는 기법.
    * 모델의 출력 logit의 모든 요솟값을 temperature로 나누는 방식. 여기에 소프트맥스를 취해 다음 단어 확률 분포로 만든다.
    * temperature은 음수면 대소관계가 뒤바뀌므로 0보다 큰 값 지정해야 함. (1이면 변화 없음)
    * `temperature`로 지정하며 1보다 큰 값으로 설정할수록 확률 분포가 평평해져 다양한 문장이 생성될 수 있다.
    * ![temperature](img/sampling_search_with_temp.png)

* 논문: [gpt-based generation for classical chinese poetry](https://arxiv.org/abs/1907.00151)
* 국내 연구: 사전 학습 언어 모델을 이용한 한국어 판결문 생성 요약(성균관대학교 컴퓨터공학과 석사학위논문)

### 1장
* 모델 => 확률을 나타내는 것 => 모델 역시 원하는 task에 맞게 선택하는 것 => 최근에는 딥러닝 기반 언어 모델이 자주 선택됨.(BERT, GPT)
* 트랜스퍼 러닝: 트랜스퍼 러닝이란 특정 태스크를 학습한 모델을 다른 태스크 수행에 재사용하는 기법이다.
* 트랜스퍼 러닝의 장점: 기존보다 모델의 학습 속도가 빨라지고 새로운 태스크를 더 잘 수행한다.
    * 업스트림 태스크: 다음 단어 맞히기, 빈칸 채우기 등 대규모 말뭉치의 문맥을 이해하는 과제
    * 다운스트림 태스크: 우리가 풀고자 하는 구체적인 과제
* 프리 트레인: 업스트림 태스크를 학습하는 과정
* 업스트림 태스크의 대표 과제
    * 다음 단어 맞히기 : 대표적으로 GPT
    * 빈칸 채우기 : BERT 계열 모델 (마스크 언어 모델, masked language model)
* 업스트림 태스크 장점 : 지도 학습의 경우 사람이 일일이 레이블링을 해야 하므로 비용 + 실수 발생 가능. 반면, 업스트림 태스크는 뉴스, 웹문서, 백과사전 등 글만 있으면 다량의 학습 데이터를 낮은 비용으로 만들 수 있음.
* 자기지도 학습 (self-supervised learning) : 데이터 내에서 정답을 만들고 이를 바탕으로 모델을 학습하는 방법
* 보통 다운스트림 태스크는 프리트레인을 마친 모델을 구조 변경 없이 그대로 사용하거나 태스크 모듈을 덧붙인 형태로 수행한다.
* 문장 생성을 제외한 대부분의 과제는 프리트레인을 마친 마스크 언어 모델(BERT 계열)을 사용한다.
* 파인 튜닝: 다운스트림 태스크의 학습 방식 중 하나. 프리트레인을 마친 모델을 다운스트림 태스크에 맞게 업데이트하는 기법이다.
    * 다운스트림 태스크 데이터 전체를 사용하여 데이터에 맞게 모델 전체를 업데이트 한다. 파인 튜닝 외에도 프롬프트 튜닝, 인컨텍스트 러닝 존재
* **문장 생성 모델은 GPT 계열 언어 모델이 널리 쓰입니다.**
* 컬레이트(collate): 배치의 모양을 정비하여 모델의 최종 입력으로 만들어 주는 과정
* 컬레이트 수행 방식 역시 파이토치 사용자가 자유롭게 구성할 수 있음.

### 2장 토큰화
* 서브워드 단어 토큰화: 단어 단위 토큰화 / 문자 단위 토큰화의 중간 형태. 어휘 집합 크기가 지나치게 커지지 않으면서도 미등록 토큰 문제를 피하고, 분석된 토큰 시퀀스가 너무 길어지지 않게 한다.
* BPE(바이트 페어 인코딩, Byte Pair Encoding): 원래 정보를 압축하던 알고리즘. GPT 모델은 BPE 기법으로 토큰화 수행 / BERT 모델은 BPE와 유사한 워드피스(wordpiece)를 토크나이저로 사용.
* BPE 기반 토큰화 기법은 분석 대상 언어에 대한 지식이 필요 없다. 자주 나타나는 문자열을 토큰으로 분석하면 되기 때문 !
* BPE 어휘 집합은 고빈도 바이그램 쌍을 병합하는 방식으로 구축한다.
* 워드 피스는 '우도'(likelhodd)를 가장 높이는 글자 쌍을 병합한다.


### 3장 트랜스포머
* 언어 모델이란 단어 시퀀스에 확률을 부여하는 모델이다.
    * 이전 단어들이 주어졌을 때 다음 단어가 나타날 확률을 부여하는 모델
* 순방향 언어 모델: 이전 단어들이 주어졌을 때 다음 단어 맞히기로 학습. GPT, ELMo
* 역방향 언어 모델: 문장 뒤부터 앞으로 계산. ELMo(두 방향 다)
* 마스크 언어 모델: 학습 문장 대상에 빈칸을 만들어 놓고 해당 빈칸에 올 단어로 적절한 단어가 무엇일지 분류하는 과정으로 학습. BERT
* 스킵-그램 모델: 단어 앞뒤에 특정 범위를 정해 두고 범위 내에 어떤 단어들이 올지 분류하는 과정으로 학습. Word2Vec
* 언어 모델은 어떤 문장이 자연스러운지 가려낼 수 있어 그 자체로 값어치가 있음. 학습 대상 언어의 풍부한 맥락을 포함하고 있다.
* 시퀀스-투-시퀀스: 특정 속성을 지닌 시퀀스를 다른 속성의 시퀀스로 변환.
* 인코더 : 소스 시퀀스 정보 압축 (인코딩) => 디코더 : 타깃 시퀀스 생성 (디코딩)
* 트랜스포머는 인코더와 디코더 입력이 주어졌을 때 정답에 해당하는 단어의 확률값을 높이는 방식으로 학습한다.
* 트랜스포머의 인코더는 트랜스포머 블록을 수십 개 쌓아서 구성한다.
* 인코더 블록의 구성 요소: 멀티 헤드 어텐션, 피드포워드 뉴럴 네트워크, 잔차 연결 및 레이어 정규화
* 디코더 블록의 구성 요소: 마스크 멀티헤드 어텐션, 멀티 헤드 어텐션, 피드포워드 뉴럴넷, 잔차 연결 및 레이어 정규화
* 멀티 헤드 어텐션 = 셀프 어텐션 (트랜스포머의 경쟁력이라고 불린다.)
    * 어텐션은 중요한 요소에 더 집중해 성능을 끌어 올리는 기법이다.
    * 입력 시퀀스 (소스 언어) 가운데 태스크 수행에 의미 있는 요소들 위주로 정보 추출
* 셀프 어텐션은 쿼리, 키, 밸류가 서로 영향을 주고 받으며 문장의 의미를 계산한다.
* 멀티 헤드 어텐션: 셀프 어텐션을 동시에 여러 번 수행

## [Study] KoGPT2와 Tokenizing

### KoGPT2 

* SKT-AI에서 GPT2 모델을 한국어 데이터로 학습하여 공개한 언어 모델.

### Tokenizing
* 크롤링한 데이터를 모델의 입력으로 사용하려면 토큰화를 진행해야 한다.
* KoGPT2 모델은 BPE 기법(Byte Pair Encoding)으로 토큰화를 수행하였다. 
* `fine-tuning` 진행할 때는 `pre-training` 과정에서 사용한 토큰화 방법과 동일하게 진행해야 한다.

<img src="bpe.png" width="500" height="500">

* BPE는 데이터에서 가장 많이 등장한 문자열을 병합하여 데이터를 압축하는 기법이다.
* BPE를 활용한 토큰화 절차
    * 어휘 집합 구축: 자주 등장하는 문자열을 병합하고 이를 어휘 집합에 추가한다. 이를 원하는 어휘 집합 크기가 될 때까지 반복한다.
    * 토큰화: 토큰화 대상 문장의 각 어절에서 어휘 집합에 있는 서브워드가 포함되었을 때 해당 서브워드를 어절에서 분리한다.